In [1]:
from konlpy.tag import Mecab
import gensim
import numpy as np
import pickle

tag = Mecab()

# 데이터 로드 

In [2]:
lines=[]
with open('./data/recipeCorpus10000.txt', 'r', encoding="utf-8") as f: 
    reader = f.readlines()
    for row in reader:
        lines.append(row)

In [3]:
lines = [l.split('\t')[1].strip() for l in lines]

In [4]:
lines[0]

'쌀을 씻어 분 정도 물에 불려둔다 멥쌀과 찹쌀을 의 비율로 사용하면 좋습니다 뚱이랑뚱 현미랑 적미랑 흑미랑 섞으면 더 예뻐요 작은뭉치 쌀을 씻어 분 정도 물에 불려둔다 개 은행은 볶아 껍질을 벗긴다 불을 중약불로 줄인다음 물엿이나 올리고당을 수저 넣고 같이 볶아주면 껍질이 쉽게 벗겨집니다 그린씨엘 끓는 물에 데치는데 국자를 둥글게 밀어주면 껍질이 벗겨져요 기름기 없는 은행이 되요 밥심은국력 볶으면서 물엿을 조금 넣어서 주걱이나 숫가락 등으로 저어주면 잘 벗겨집니다 그린씨엘 개 밤은 껍질을 벗기고 대추는 깨끗이 씻어둔다 밤은 미리 껍질을 벗기면 색이 변해요 물에 담가 두세요 애빼시 대추는 나중에 이 상하지 않게 씨를 빼놓으세요 생강 대추는 솔로 구석 구석 닦아주셔야 합니다 개 쌀과 밤을 넣고 밥을 짓다가 한소끔 끓어오르면 대추 은행 잣을 넣고 약한 불로 뜸을 들인다 압력밥솥 등을 사용하면 은행 잣은 같이 넣고 밥을 지어도 괜찮습니다 뚱이랑뚱 다시마 한조각을 넣어 같이 밥을 지어도 좋아요 곰택 대추는 먹기좋게 씨를 제거한 뒤에 넣어주세요 피안화 개 쓰기레시피 출처 월간 헬스조선 대추나 잣 호두 등의 견과류는 오랫동안 익힐 필요가 없다 대추는 무르거나 퍼질 수 있으므로 밥을 다 한 뒤 뜸들일 때 넣는다'

In [5]:
len(lines)

11704

# 토크나이징, POS 태깅

In [6]:
result=[]
for sent in lines:
    pos = tag.pos(sent)
    pos = [(w,t) for w,t in pos if t not in ['SF', 'SE', 'SSO', 'SSC', 'SC','SY','SH','SN']]
    result.append(pos)

# 전체 토큰 

In [8]:
flatten = [word for word in sent for sent in result]

In [9]:
len(flatten)

4190032

# 토큰셋(중복제거) 

In [10]:
checker=[]
tokens = []
for sentence in result:
    for word in sentence:
        if word[0] not in checker:
            tokens.append(word)
            checker.append(word[0])
        else:
            continue

In [17]:
assert len(tokens)==len(set(tokens))

In [18]:
len(set(tokens))

21542

# 트레이닝 with gensim 

In [19]:
train_set = [[word[0] for word in sent] for sent in result]

In [22]:
model = gensim.models.Word2Vec(train_set, size=100, window=5, min_count=5, workers=4)

In [24]:
model.most_similar('김치')

[('은지', 0.6462485790252686),
 ('묵', 0.6373645067214966),
 ('배추김치', 0.5908721089363098),
 ('콩나물', 0.5432823896408081),
 ('숙주', 0.5427047610282898),
 ('배추', 0.5270323157310486),
 ('무', 0.5026322603225708),
 ('야채', 0.498574435710907),
 ('부추', 0.4957957863807678),
 ('양배추', 0.4836048185825348)]

In [25]:
model.save('./model/recipe_word.model')

# 토큰-임베딩 

In [26]:
vocabs=[]
vectors=[]
for token in tokens:
    try:
        vectors.append(model[token[0]]) # 모델에 속하는 토큰만
        vocabs.append(token)
    except:
        continue

In [27]:
print(vocabs[0],vectors[0])

('쌀', 'NNG') [-2.04397917  0.96544164 -0.14516188  1.50539935 -1.05788374  0.44640496
  0.18749818  0.11738808 -0.75119454  0.89823908  0.06348223  1.05034459
 -0.27899766 -0.29018766  0.60359925  0.91184515  1.57913589 -0.53638148
  0.45544636  0.05749752 -1.10168087 -0.41091633  0.80956733  0.55120158
  0.20780084 -1.29290891  0.56177211  0.54327834 -0.26287711 -2.49288464
  0.70417303 -1.05847371  0.48173672  0.95853168  0.21826182  0.50210047
  0.08014657  1.57846808 -2.16626525  0.26240197  2.14893842  0.00462182
 -0.76224536 -0.2804856  -0.68401283  0.57711178 -0.12922218  0.04042026
  0.59521914 -0.61784685 -0.59830791 -1.51832211 -0.41874015  1.43858266
  1.24338293  0.32604814 -1.15758073 -0.09522968 -1.17174768 -1.62155104
 -0.3288025  -0.94366229  0.57225376 -1.33341432 -0.53292251 -1.39110339
  0.83840698  0.02797502  0.69429368 -1.93177652  0.65214753  2.32315993
 -0.96666676  0.64768177  2.43960953 -0.30009192  0.0061374  -0.5679965
 -1.30592763 -0.00834877  0.75775582  0

In [32]:
embeddings = np.vstack(vectors)

In [33]:
len(embeddings)

9564

In [34]:
embeddings.shape

(9564, 100)

In [35]:
len(vocabs)

9564

In [36]:
pickle.dump(embeddings, open('./data/embedding.p','wb'))
pickle.dump(vocabs, open('./data/vocab.p','wb'))